In [1]:
# notebooks/02_active_loop_simulation.ipynb

# %% [markdown]
# # 02. Active Loop & Evaluation
# 
# ## Goal
# Simulate the game session. For every transcript chunk:
# 1. Search the **Rulebook** for matches.
# 2. Search the **Campaign** for matches.
# 3. If similarity is high, ask **Gemini**.
# 4. Save all data to a Table for review.

# %%
import os
import sys
import time
from datetime import datetime
import pandas as pd

# Langchain imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Import project config & utils
sys.path.append(os.path.abspath('../src'))
from config import (
    VECTOR_STORE_DIR, EMBEDDING_MODEL_PATH, 
    CHUNK_SIZE, OVERLAP, TRIGGER_THRESHOLD, TOP_K
)
from utils import clean_transcript_text, get_sliding_windows
# Import the new Factory
from llm_factory import get_llm


c:\Users\pfeil\anaconda3\envs\dm_copilot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# %% [markdown]
# ## Step 1: Initialize Models (Clean)

# %%
# 1. Load Local Fine-Tuned Embeddings
print(f"🧠 Loading Embeddings from: {EMBEDDING_MODEL_PATH}...")
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_PATH,
    model_kwargs={"trust_remote_code": True, "device": "cpu"}
)

# 2. Load Vector DB
print("💾 Loading Database...")
vectorstore = Chroma(persist_directory=VECTOR_STORE_DIR, embedding_function=embedding_model)

# 3. Get LLM from Factory (Logic hidden in src/llm_factory.py)
llm = get_llm()


🧠 Loading Embeddings from: c:\Users\pfeil\My Drive\Studys\CAS-NLP-Uni-Bern\Module 4  NLP Transformers\dungeon_master_copilot\Output\fine_tuned_qwen_dnd...


C:\Users\pfeil\AppData\Local\Temp\ipykernel_26340\3858769005.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
The tokenizer you are loading from 'c:\Users\pfeil\My Drive\Studys\CAS-NLP-Uni-Bern\Module 4  NLP Transformers\dungeon_master_copilot\Output\fine_tuned_qwen_dnd' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
C:\Users\pfeil\AppData\Local\Temp\ipykernel_26340\3858769005.py:14: LangChainDeprecation

💾 Loading Database...
🏭 Factory Initializing LLM: OLLAMA


In [3]:
# %% [markdown]
# ## Step 2: Define Retrieval & Chain

# %%
prompt_template = """
You are a Dungeon Master's Assistant.
Here is the live conversation which comes from the Dungeon Master or from a player: "{transcript_chunk}"

Rules Context:
{rule_context}

Campaign Context:
{campaign_context}

Task:
1. Does the conversation require a specific fact from the Rules or Campaign to help the Dungeon Master?
2. If YES, provide the fact concisely.
3. If NO (it's just chit-chat or roleplay), respond exactly "NO_ACTION".

Answer:
"""

prompt = PromptTemplate.from_template(prompt_template)
rag_chain = prompt | llm | StrOutputParser()

def get_best_matches(query, source_type, k=TOP_K):
    """
    Returns list of (Document, score) tuples.
    """
    return vectorstore.similarity_search_with_score(
        query, k=k, filter={"source_type": source_type}
    )

def format_matches_for_log(matches):
    """
    Helper to format all K matches into a readable string for the CSV.
    Format: [Score: 0.45] Text snippet... | [Score: 0.52] Text snippet...
    """
    if not matches:
        return "None"
    
    log_entries = []
    for doc, score in matches:
        # Round score and take first 100 chars of text to keep CSV readable
        entry = f"[Score: {score:.4f}] {doc.page_content[:150]}..." 
        log_entries.append(entry)
    return " | ".join(log_entries)


In [4]:
# %% [markdown]
# ## Step 3: Run Simulation with Timestamps

# %%
TRANSCRIPT_FILE = "../data/transcripts/live-play-transcript.txt"
clean_text = clean_transcript_text(TRANSCRIPT_FILE)
chunks = list(get_sliding_windows(clean_text, CHUNK_SIZE, OVERLAP))

print(f"🚀 Starting Simulation on {len(chunks)} chunks...")
eval_data = []

total_start = time.time()

for i, chunk in enumerate(chunks):
    print(f"Processing Chunk {i+1}/{len(chunks)}...", end="\r")
    
    # 1. Capture Start Time
    ts_start = datetime.now().strftime("%H:%M:%S.%f")[:-3]
    start_perf = time.perf_counter() # For calculation
    
    # --- A. Retrieval ---
    rule_matches = get_best_matches(chunk, "rulebook", k=TOP_K)
    camp_matches = get_best_matches(chunk, "campaign", k=TOP_K)
    
    # Get best scores (Default to 1.0 if no match found)
    best_rule_score = rule_matches[0][1] if rule_matches else 1.0
    best_camp_score = camp_matches[0][1] if camp_matches else 1.0
    
    # Trigger Logic
    is_triggered = (best_rule_score < TRIGGER_THRESHOLD) or (best_camp_score < TRIGGER_THRESHOLD)
    
    llm_response = "SKIPPED"
    
    # --- B. LLM Generation ---
    if is_triggered:
        # Create Context Strings (using top match for simplicity in context, but logging all)
        rule_txt = "\n".join([d.page_content for d, s in rule_matches]) if best_rule_score < TRIGGER_THRESHOLD else "No relevant rules."
        camp_txt = "\n".join([d.page_content for d, s in camp_matches]) if best_camp_score < TRIGGER_THRESHOLD else "No relevant campaign facts."

        try:
            response = rag_chain.invoke({
                "transcript_chunk": chunk,
                "rule_context": rule_txt,
                "campaign_context": camp_txt
            })
            llm_response = response.strip()
        except Exception as e:
            llm_response = f"ERROR: {str(e)}"
    
    # 2. Capture End Time
    ts_end = datetime.now().strftime("%H:%M:%S.%f")[:-3]
    end_perf = time.perf_counter()
    duration = round(end_perf - start_perf, 3)
    
    # --- C. Log Detailed Data ---
    row = {
        "Chunk ID": i,
        "Start Time": ts_start,
        "End Time": ts_end,
        "Duration (s)": duration,
        "Transcript": chunk,
        "Triggered?": is_triggered,
        "LLM Response": llm_response,
        # Rule Data
        "Best Rule Score": round(best_rule_score, 4),
        "All Rule Matches (Top K)": format_matches_for_log(rule_matches),
        # Campaign Data
        "Best Camp Score": round(best_camp_score, 4),
        "All Camp Matches (Top K)": format_matches_for_log(camp_matches),
    }
    eval_data.append(row)

print(f"\n✅ Simulation Complete.")

🚀 Starting Simulation on 70 chunks...
Processing Chunk 70/70...
✅ Simulation Complete.


In [5]:
# %% [markdown]
# ## Step 4: Evaluation & Visualization

# %%
df = pd.DataFrame(eval_data)
active_rows = df[df["Triggered?"] == True]

pd.set_option('display.max_colwidth', None)

print(f"📊 Processed {len(df)} chunks.")
print(f"⏱️ Average Processing Time: {df['Duration (s)'].mean():.3f}s")

# Display key columns including the detailed matches
cols_to_show = [
    "Start Time", 
    "Duration (s)", 
    "Transcript", 
    "LLM Response",
    "All Rule Matches (Top K)",   # Added this
    "All Camp Matches (Top K)"    # Added this
]

# We display the head of the active rows with the new columns
display(active_rows[cols_to_show].head())

# Save full report (This will automatically contain ALL columns created in Step 3)
df.to_csv("simulation_results_detailed.csv", index=False)
print("✅ Saved detailed results to simulation_results_detailed.csv")

📊 Processed 70 chunks.
⏱️ Average Processing Time: 1.909s


,Start Time,Duration (s),Transcript,LLM Response,All Rule Matches (Top K),All Camp Matches (Top K)
3,21:19:39.647,17.898,flan it's the name of the city here and flan so far that has been true madam free ona is a stout and officious halfling who runs the establishment with her five daughters and has proven to be an excellent hostess it's quite cozy although you have had to share a common bunk room with several other adventurers the evening meal was excellent and the atmosphere is pleasant somebody left,"1. Yes, the conversation requires a specific fact from the Rules: Defiance in phlan #6.\n\n2. No_action","[Score: 1.0019] Alignment. Most halflings are lawful good. As a rule, they are good-hearted and kind, hate to\nsee others in pain, and have no tolerance for oppression... | [Score: 1.0450] Comfortable. Choosing a comfortable lifestyle means that you can afford nicer clothing and can\neasily maintain your equipment. You live in a small cot... | [Score: 1.1916] the consequences.\nA disease that does more than infect a few party members is primarily a plot device. The rules\nhelp describe the effects of the dise...",[Score: 0.6671] Defiance in phlan \n Not for resale. Permission granted to print or photocopy this document for personal use only. 23 \nMission 4: A Shock at \nEvenf... | [Score: 0.6993] Defiance in phlan \n Not for resale. Permission granted to print or photocopy this document for personal use only. 6 \nMission 1: The Meeting \nat De... | [Score: 0.7506] Defiance in phlan \n Not for resale. Permission granted to print or photocopy this document for personal use only. 11 \n \nMission 2: The Screams \nat...
5,21:19:58.574,14.099,preparing to bunk down for the evening when one of free owners daughters peeks into the room she calls a few of the adventures including all of you into the hall pardon my interruption says the halfling girl named Rhys a chap just came into the common room downstairs and he asked me to fetch you something about some coin needing to change hands for an easy job she plays,"1. The conversation requires the following specific fact from the Rules or Campaign:\n\n* Defiance in phlan: Not for resale. Permission granted to print or photocopy this document for personal use only.\n\n2. Since there is no relevant rule or campaign context provided, I will respond with a general answer:\n\nNO_ACTION","[Score: 1.1079] unoccupied space within range. If you know a specific creature’s name, you can speak that\nname when you cast this spell to request that creature, thou... | [Score: 1.1157] Alignment. Most halflings are lawful good. As a rule, they are good-hearted and kind, hate to\nsee others in pain, and have no tolerance for oppression... | [Score: 1.1438] complex (spy on our enemies, or protect us during our foray into the dungeon). You must be\nable to communicate with the creature to bargain for its se...",[Score: 0.6368] Defiance in phlan \n Not for resale. Permission granted to print or photocopy this document for personal use only. 6 \nMission 1: The Meeting \nat De... | [Score: 0.8028] Defiance in phlan \n Not for resale. Permission granted to print or photocopy this document for personal use only. 11 \n \nMission 2: The Screams \nat... | [Score: 0.9762] Defiance in phlan \n Not for resale. Permission granted to print or photocopy this document for personal use only. 23 \nMission 4: A Shock at \nEvenf...
12,21:20:18.764,14.817,you know either a human or elf but you can't really tell his identity or anything else he speaks in a low voice and he begins to talk he says I'm a member of the Harper's and I have a job for you what's that job well you see we captured a merchant who was going to illegally purchase a red dragon egg I want you guys to pose as,1. The conversation requires a specific fact from the Rules: DC 10 Charisma (Deception) check and DC 10 Dexterity (Sleight of Hand) check.\n\n2. NO_ACTION,"[Score: 1.0736] through years of practice. After resting in this way, you gain the same benefit th

✅ Saved detailed results to simulation_results_detailed.csv
